In [1]:
from Pneumotracker_func import get_and_create_dirs, build_df, build_train_model, model_smpl, model_comp, segment_image, dice_coef_loss, dice_coef
# from Segmentation_func import 

import os
import pandas as pd
from keras.models import load_model
from tqdm import tqdm

orig_path, seg_path = r'D:\MOOCS\DataScientest\Projet\Dataset\chest_xray\chest_xray', r'D:\MOOCS\DataScientest\Projet\UNET\segmentation'
orig_file_ext = 'jpeg'
smpl_checkp_name = r'.\Models\model_simple_orig_224_rgb.h5'
smpl_hist_name = r'.\Models\history_simple_orig_224_rgb.csv'
seg_model = r'.\Models\unet_lung_seg.hdf5'
cmpl_orig_checkp_name = r'.\Models\model_complex_orig_224_rgb.h5'
cmpl_orig_hist_name = r'.\Models\history_complex_orig_224_rgb.csv'
cmpl_seg_checkp_name = r'.\Models\model_complex_seg_224_rgb.h5'
cmpl_seg_hist_name = r'.\Models\history_complex_seg_224_rgb.csv'

## Intialize variables

In [2]:
orig_path, seg_path = get_and_create_dirs()
orig_file_ext = input('\nWhat is the file extension for original images ?\n').replace('.', '')
seg_model = input('\nWhat is the path to the segmentation model checkpoint ?\n')

Checks for paths for original and segmented images.
Original and segmented images must be organized in the exact same structure (images in the same folders and sub-folders).

Input root path for original images:
D:\MOOCS\DataScientest\Projet\Dataset\chest_xray\chest_xray

Input root path for segmented images if exists or needs creation:
D:\MOOCS\DataScientest\Projet\UNET\segmentation

What is the file extension for original images ?
jpeg

What is the path to the segmentation model checkpoint ?
unet_lung_seg.hdf5


In [3]:
# D:\MOOCS\DataScientest\Projet\Dataset\chest_xray\chest_xray
# D:\MOOCS\DataScientest\Projet\UNET\segmentation
# unet_lung_seg.hdf5

## Build dataset DataFrame

In [4]:
print('Building dataset DataFrame')

df = build_df(path_orig = orig_path,
              orig_img_ext = 'jpeg',
              path_seg = seg_path,
              seg_img_ext = 'png')

100%|██████████| 9/9 [00:00<?, ?it/s]

Building dataset DataFrame
Done


#### Necessary to ensure same data split between file systems

In [5]:
df = pd.read_csv('df_all.csv', index_col = 0)
df['Filepath_orig'] = df['Filepath_orig'].apply(lambda x: x.replace('../input/chest-xray-pneumonia/chest_xray/chest_xray', orig_path).replace("/", "\\"))
df['Filepath_seg'] = df['Filepath_seg'].apply(lambda x: x.replace('../input/segmentation-finale/segmentation', seg_path).replace("/", "\\"))
df.head()

,Label_name,Label_int,Label_pathology,Label_pathology_int,Filename_orig,Filepath_orig,Filename_seg,Filepath_seg
0,PNEUMONIA,1,bacteria,1,person1947_bacteria_4876.jpeg,D:\MOOCS\DataScientest\Projet\Dataset\chest_xr...,person1947_bacteria_4876.png,D:\MOOCS\DataScientest\Projet\UNET\segmentatio...
1,PNEUMONIA,1,bacteria,1,person1946_bacteria_4875.jpeg,D:\MOOCS\DataScientest\Projet\Dataset\chest_xr...,person1946_bacteria_4875.png,D:\MOOCS\DataScientest\Projet\UNET\segmentatio...
2,PNEUMONIA,1,bacteria,1,person1952_bacteria_4883.jpeg,D:\MOOCS\DataScientest\Projet\Dataset\chest_xr...,person1952_bacteria_4883.png,D:\MOOCS\DataScientest\Projet\UNET\segmentatio...
3,PNEUMONIA,1,bacteria,1,person1954_bacteria_4886.jpeg,D:\MOOCS\DataScientest\Projet\Dataset\chest_xr...,person1954_bacteria_4886.png,D:\MOOCS\DataScientest\Projet\UNET\segmentatio...
4,PNEUMONIA,1,bacteria,1,person1951_bacteria_4882.jpeg,D:\MOOCS\DataScientest\Projet\Dataset\chest_xr...,person1951_bacteria_4882.png,D:\MOOCS\DataScientest\Projet\UNET\segmentatio...


## Building and training simple CNN

In [14]:
model_smpl.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 220, 220, 30)      780       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 30)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 108, 108, 16)      4336      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 54, 54, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 46656)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               5

In [8]:
# smpl_checkp_name = r'.\Models\model_simple_orig_224_grayscale.h5'
# smpl_hist_name = r'.\Models\history_simple_orig_224_grayscale.csv'

Simple model checkpoint found. Do you want to retrain model (Y/N)?N
False


In [6]:
smpl_checkp_name = input('Input path for simple model checkpoint (XXXX.h5).\n')
smpl_hist_name = input('Input path for simple model history (XXXX.csv).\n')

train = False

if 'smpl_checkp_name' in locals():
    if os.path.exists(smpl_checkp_name):
        train = True if input('Simple model checkpoint found. Do you want to retrain model (Y/N)?') == 'Y' else False
    else:
        train = True
else:
    train = True    

X = df['Filepath_orig']
Y = df['Label_name']

md_smpl_eval, history_smpl, smpl_model_class_rep, smpl_model_conf_mat, smpl_model = build_train_model(X, Y, model_smpl, 2, 32, 30, smpl_checkp_name, smpl_hist_name, train)

Input path for simple model checkpoint (XXXX.h5).
sdqdsqdqsdqdqs.h5
Input path for simple model history (XXXX.csv).
qsdqdsqdqds.csv

Splitting data

Building Data Generators
Found 4099 validated image filenames belonging to 2 classes.
Found 878 validated image filenames belonging to 2 classes.
Found 879 validated image filenames belonging to 2 classes.

Computing Class weights
Weight for class NORMAL : 1.85
Weight for class PNEUMONIA : 0.69

Building callbacks:
Checkpoint
Reduce learning rate on plateau
Early stopping for val_loss

Building model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 220, 220, 30)      2280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 30)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (N

KeyboardInterrupt: 

In [5]:
smpl_model_conf_mat

Predicted,NORMAL,PNEUMONIA
Real,,
NORMAL,215,23
PNEUMONIA,19,622


In [2]:
segmentation_model = load_model(seg_model, 
                                custom_objects={'dice_coef_loss': dice_coef_loss,
                                                'dice_coef': dice_coef})

In [3]:
segmentation_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 512, 512, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 256, 256, 32) 0           conv2d_2[0][0]                   
____________________________________________________________________________________________

In [8]:
for dirname, _, filenames in os.walk(orig_path):
    for filename in tqdm(filenames):
        if ('.' + orig_file_ext) in filename:
            segment_image(segmentation_model, os.path.join(dirname, filename), dirname.replace(orig_path, seg_path))

0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/234 [00:00<?, ?it/s]

IM-0001-0001.jpeg


  0%|          | 1/234 [00:00<03:34,  1.09it/s]

IM-0003-0001.jpeg


  1%|          | 2/234 [00:01<03:04,  1.26it/s]

...

IM-0007-0001.jpeg


  2%|▏         | 5/234 [00:03<02:46,  1.38it/s]

IM-0009-0001.jpeg


  2%|▏         | 5/234 [00:04<03:24,  1.12it/s]


KeyboardInterrupt: 

In [6]:
X = df['Filepath_orig']
Y = df['Label_name']

md_comp_orig_eval, history_comp_orig = build_train_model(X, Y, model_smpl, 2, 32, 30, 'model_comp_orig_224_rgb.h5', 'history_comp_orig_224_rgb')

0
1


In [ ]:
# Créer les répertoires pour la segmentation
# Segmenter les images

In [ ]:
X = df['Filepath_seg']
Y = df['Label_name']

md_comp_orig_eval, history_comp_orig = build_train_model(X, Y, model_smpl, 2, 32, 30, 'model_comp_orig_224_rgb.h5', 'history_comp_orig_224_rgb')

In [4]:
# smpl_checkp_name = input('Input path for simple model checkpoint.\n')
# smpl_hist_name = input('Input path for simple model history.\n')

train = False

if 'smpl_checkp_name' in locals():
    if os.path.exists(smpl_checkp_name):
        train = True if input('Simple model checkpoint found. Do you want to retrain model (Y/N)?') == 'Y' else False
    else:
        train = True
else:
    train = True    

X = df['Filepath_orig']
Y = df['Label_name']

md_smpl_eval, history_smpl, smpl_model_class_rep, smpl_model_conf_mat, smpl_model = build_train_model(X, Y, model_comp, 2, 32, 30, cmpl_orig_checkp_name, cmpl_orig_hist_name, train)

Simple model checkpoint found. Do you want to retrain model (Y/N)?N

Splitting data

Building Data Generators
Found 4099 validated image filenames belonging to 2 classes.
Found 878 validated image filenames belonging to 2 classes.
Found 879 validated image filenames belonging to 2 classes.

Evaluating model
28/28 [==============================] - 16s 573ms/step - loss: 0.0620 - accuracy: 0.9898
Loss:  0.06203903257846832
Accuracy:  98.97611141204834 %


AttributeError: 'numpy.ndarray' object has no attribute 'apply'